In [67]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [68]:
rr = RandomRules(
  TOTAL_NUMBERS = 35,
  CHOOSE = 5,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5],
  MAX_B_VALUES = [31, 32, 33, 34, 35]
)

In [69]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=164'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [70]:
lst = []
post_str_b1 = None
post_str_blast = None
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=7728


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""12/04/2025""","""4184""","""23 12 10 16 03""","""0310121623""","[3, 10, … 23]",3,10,12,16,23,"""03""","""10""","""12""","""16""","""23""",null,null,false,0,"""07_02_04_07""",-6
"""11/04/2025""","""4183""","""09 10 08 01 12""","""0108091012""","[1, 8, … 12]",1,8,9,10,12,"""01""","""08""","""09""","""10""","""12""","""03""","""23""",true,24,"""07_01_01_02""",-5
"""10/04/2025""","""4182""","""30 33 34 10 21""","""1021303334""","[10, 21, … 34]",10,21,30,33,34,"""10""","""21""","""30""","""33""","""34""","""01""","""12""",false,0,"""11_09_03_01""",8
"""09/04/2025""","""4181""","""29 21 20 28 04""","""0420212829""","[4, 20, … 29]",4,20,21,28,29,"""04""","""20""","""21""","""28""","""29""","""10""","""34""",true,16,"""16_01_07_01""",-33
"""08/04/2025""","""4180""","""10 20 30 05 16""","""0510162030""","[5, 10, … 30]",5,10,16,20,30,"""05""","""10""","""16""","""20""","""30""","""04""","""29""",false,0,"""05_06_04_10""",11
"""07/04/2025""","""4179""","""08 21 22 24 07""","""0708212224""","[7, 8, … 24]",7,8,21,22,24,"""07""","""08""","""21""","""22""","""24""","""05""","""30""",true,16,"""01_13_01_02""",37
"""06/04/2025""","""4178""","""07 35 24 34 14""","""0714243435""","[7, 14, … 35]",7,14,24,34,35,"""07""","""14""","""24""","""34""","""35""","""07""","""24""",true,8,"""07_10_10_01""",-6
"""05/04/2025""","""4177""","""34 33 15 05 16""","""0515163334""","[5, 15, … 34]",5,15,16,33,34,"""05""","""15""","""16""","""33""","""34""","""07""","""35""",true,16,"""10_01_17_01""",-57
"""04/04/2025""","""4176""","""27 24 32 30 35""","""2427303235""","[24, 27, … 35]",24,27,30,32,35,"""24""","""27""","""30""","""32""","""35""","""05""","""34""",true,24,"""03_03_02_03""",3


In [71]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str):
  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )
  mean_percent = df_viz['percent'].mean()
  mean_percent_75 = mean_percent * 75 / 50
  mean_percent_25 = mean_percent * 25 / 50

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart = chart * hv.HLine(mean_percent_75).opts(color='blue')
  chart = chart * hv.HLine(mean_percent_25).opts(color='blue')
  
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  df_unijump = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'unijump'
    ).agg(pl.count('unijump').alias('count')) \
    .sort("count", descending=True)


  print(f'df_viz FOR {col} = {test_value}')
  display(df_viz)

  display(chart)
  
  print(f'df_10_last FOR {col} = {test_value}')
  display(df_10_last)
  
  print(f'df_jumps_map FOR {col} = {test_value}')
  display(df_jumps_map)
  # display(df_unijump)

  print(f'df_unijump FOR {col} = {test_value}')
  display(df_unijump)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list() # Used to discard
  
  return lst_post_str_b, lst_jumps_map

In [72]:
possible_b1, exclude_jump_maps_b1 = plot_post_col(col='str_b1', post_col='post_str_b1', test_value="03")

df_viz FOR str_b1 = 03


post_str_b1,count,percent
str,u32,f64
"""01""",120,14.268728
"""02""",119,14.149822
"""03""",99,11.7717
"""04""",78,9.274673
"""05""",70,8.323424
…,…,…
"""23""",1,0.118906
"""20""",1,0.118906
"""24""",1,0.118906


:Overlay
   .Bars.I    :Bars   [post_str_b1]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b1 = 03


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""30/03/2025""","""4171""","""35 05 15 29 03""","""0305152935""","[3, 5, … 35]",3,5,15,29,35,"""03""","""05""","""15""","""29""","""35""","""07""","""34""",true,2,"""02_10_14_06""",-8
"""14/03/2025""","""4155""","""35 11 03 07 15""","""0307111535""","[3, 7, … 35]",3,7,11,15,35,"""03""","""07""","""11""","""15""","""35""","""02""","""27""",true,26,"""04_04_04_20""",16
"""12/03/2025""","""4153""","""28 14 35 03 15""","""0314152835""","[3, 14, … 35]",3,14,15,28,35,"""03""","""14""","""15""","""28""","""35""","""07""","""30""",false,0,"""11_01_13_07""",-40
"""07/03/2025""","""4148""","""26 07 03 24 30""","""0307242630""","[3, 7, … 30]",3,7,24,26,30,"""03""","""07""","""24""","""26""","""30""","""02""","""35""",true,16,"""04_17_02_04""",45
"""27/02/2025""","""4140""","""14 29 03 04 07""","""0304071429""","[3, 4, … 29]",3,4,7,14,29,"""03""","""04""","""07""","""14""","""29""","""04""","""33""",false,0,"""01_03_07_15""",2
"""22/02/2025""","""4135""","""34 33 15 24 03""","""0315243334""","[3, 15, … 34]",3,15,24,33,34,"""03""","""15""","""24""","""33""","""34""","""11""","""32""",true,8,"""12_09_09_01""",-11
"""26/01/2025""","""4108""","""09 35 03 30 32""","""0309303235""","[3, 9, … 35]",3,9,30,32,35,"""03""","""09""","""30""","""32""","""35""","""02""","""26""",true,16,"""06_21_02_03""",54
"""24/01/2025""","""4106""","""07 03 28 15 31""","""0307152831""","[3, 7, … 31]",3,7,15,28,31,"""03""","""07""","""15""","""28""","""31""","""06""","""31""",false,0,"""04_08_13_03""",-16
"""20/01/2025""","""4102""","""19 03 34 25 04""","""0304192534""","[3, 4, … 34]",3,4,19,25,34,"""03""","""04""","""19""","""25""","""34""","""21""","""34""",false,0,"""01_15_06_09""",35


df_jumps_map FOR str_b1 = 03


jumps_map,count
str,u32
"""11_03_11_05""",2
"""11_12_01_02""",2
"""20_07_02_03""",2
"""04_04_04_20""",2
"""02_19_07_02""",2
…,…
"""02_06_09_13""",1
"""01_01_22_05""",1
"""09_06_06_11""",1


df_unijump FOR str_b1 = 03


unijump,count
i64,u32
-2,24
16,19
6,18
-8,17
7,17
…,…
64,1
-63,1
46,1


In [73]:
possible_blast, exclude_jump_maps_blast = plot_post_col(col='str_b5', post_col='post_str_blast', test_value="23")

df_viz FOR str_b5 = 23


post_str_blast,count,percent
str,u32,f64
"""35""",28,17.610063
"""33""",20,12.578616
"""31""",16,10.062893
"""32""",14,8.805031
"""34""",14,8.805031
…,…,…
"""22""",4,2.515723
"""19""",3,1.886792
"""23""",2,1.257862


:Overlay
   .Bars.I    :Bars   [post_str_blast]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b5 = 23


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""29/03/2025""","""4170""","""23 06 04 09 10""","""0406091023""","[4, 6, … 23]",4,6,9,10,23,"""04""","""06""","""09""","""10""","""23""","""03""","""35""",false,0,"""02_03_01_13""",17
"""12/01/2025""","""4094""","""16 07 23 18 04""","""0407161823""","[4, 7, … 23]",4,7,16,18,23,"""04""","""07""","""16""","""18""","""23""","""08""","""28""",false,0,"""03_09_02_05""",23
"""04/12/2024""","""4055""","""05 06 11 16 23""","""0506111623""","[5, 6, … 23]",5,6,11,16,23,"""05""","""06""","""11""","""16""","""23""","""05""","""24""",true,8,"""01_05_05_07""",6
"""15/11/2024""","""4036""","""16 23 12 21 14""","""1214162123""","[12, 14, … 23]",12,14,16,21,23,"""12""","""14""","""16""","""21""","""23""","""01""","""26""",true,8,"""02_02_05_02""",-9
"""22/09/2024""","""3982""","""06 23 16 14 02""","""0206141623""","[2, 6, … 23]",2,6,14,16,23,"""02""","""06""","""14""","""16""","""23""","""08""","""32""",false,0,"""04_08_02_07""",21
"""02/09/2024""","""3962""","""07 05 09 18 23""","""0507091823""","[5, 7, … 23]",5,7,9,18,23,"""05""","""07""","""09""","""18""","""23""","""14""","""33""",true,8,"""02_02_09_05""",-18
"""07/07/2024""","""3905""","""17 08 15 23 02""","""0208151723""","[2, 8, … 23]",2,8,15,17,23,"""02""","""08""","""15""","""17""","""23""","""17""","""33""",false,0,"""06_07_02_06""",15
"""17/04/2024""","""3824""","""12 20 01 23 05""","""0105122023""","[1, 5, … 23]",1,5,12,20,23,"""01""","""05""","""12""","""20""","""23""","""03""","""33""",false,0,"""04_07_08_03""",-4
"""15/03/2024""","""3791""","""15 23 14 19 02""","""0214151923""","[2, 14, … 23]",2,14,15,19,23,"""02""","""14""","""15""","""19""","""23""","""05""","""31""",true,8,"""12_01_04_04""",-17


df_jumps_map FOR str_b5 = 23


jumps_map,count
str,u32
"""05_01_09_03""",3
"""04_02_14_01""",1
"""18_01_01_02""",1
"""06_02_05_04""",1
"""02_01_04_05""",1
…,…
"""09_03_01_01""",1
"""04_06_06_02""",1
"""05_01_09_06""",1


df_unijump FOR str_b5 = 23


unijump,count
i64,u32
5,8
-18,7
10,7
-4,6
-1,6
…,…
40,1
-28,1
11,1


In [74]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [75]:
lst = []
post_str_b1 = None
post_str_blast = None
for row in combos:
  cols = []
  str_row = " ".join([str(r).zfill(2) for r in row])
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

df_Orows = pl.DataFrame(
  lst,
  schema=[
    # 'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(df_Orows)
print(f'{TOTAL_LINES=}')
display(df_Orows.head(10))

TOTAL_LINES=324632


id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0102030405""","[1, 2, … 5]",1,2,3,4,5,"""01""","""02""","""03""","""04""","""05""",null,null,true,8,"""01_01_01_01""",0
"""0102030406""","[1, 2, … 6]",1,2,3,4,6,"""01""","""02""","""03""","""04""","""06""","""01""","""05""",true,8,"""01_01_01_02""",1
"""0102030407""","[1, 2, … 7]",1,2,3,4,7,"""01""","""02""","""03""","""04""","""07""","""01""","""06""",true,8,"""01_01_01_03""",2
"""0102030408""","[1, 2, … 8]",1,2,3,4,8,"""01""","""02""","""03""","""04""","""08""","""01""","""07""",true,8,"""01_01_01_04""",3
"""0102030409""","[1, 2, … 9]",1,2,3,4,9,"""01""","""02""","""03""","""04""","""09""","""01""","""08""",true,8,"""01_01_01_05""",4
"""0102030410""","[1, 2, … 10]",1,2,3,4,10,"""01""","""02""","""03""","""04""","""10""","""01""","""09""",true,8,"""01_01_01_06""",5
"""0102030411""","[1, 2, … 11]",1,2,3,4,11,"""01""","""02""","""03""","""04""","""11""","""01""","""10""",true,8,"""01_01_01_07""",6
"""0102030412""","[1, 2, … 12]",1,2,3,4,12,"""01""","""02""","""03""","""04""","""12""","""01""","""11""",true,8,"""01_01_01_08""",7
"""0102030413""","[1, 2, … 13]",1,2,3,4,13,"""01""","""02""","""03""","""04""","""13""","""01""","""12""",true,8,"""01_01_01_09""",8


In [76]:
df_Orows.filter(is_rare_combination=False)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0102040513""","[1, 2, … 13]",1,2,4,5,13,"""01""","""02""","""04""","""05""","""13""","""01""","""12""",false,0,"""01_02_01_08""",10
"""0102040523""","[1, 2, … 23]",1,2,4,5,23,"""01""","""02""","""04""","""05""","""23""","""01""","""22""",false,0,"""01_02_01_18""",20
"""0102040530""","[1, 2, … 30]",1,2,4,5,30,"""01""","""02""","""04""","""05""","""30""","""01""","""29""",false,0,"""01_02_01_25""",27
"""0102040531""","[1, 2, … 31]",1,2,4,5,31,"""01""","""02""","""04""","""05""","""31""","""01""","""30""",false,0,"""01_02_01_26""",28
"""0102040532""","[1, 2, … 32]",1,2,4,5,32,"""01""","""02""","""04""","""05""","""32""","""01""","""31""",false,0,"""01_02_01_27""",29
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2829313435""","[28, 29, … 35]",28,29,31,34,35,"""28""","""29""","""31""","""34""","""35""","""28""","""35""",false,0,"""01_02_03_01""",-3
"""2830313334""","[28, 30, … 34]",28,30,31,33,34,"""28""","""30""","""31""","""33""","""34""","""28""","""35""",false,0,"""02_01_02_01""",-4
"""2830313435""","[28, 30, … 35]",28,30,31,34,35,"""28""","""30""","""31""","""34""","""35""","""28""","""35""",false,0,"""02_01_03_01""",-7


In [77]:
# possible_b1, exclude_jump_maps_b1

df_possible = df_Orows.filter(
  # pl.col("str_b1").is_in(possible_b1),
  pl.col("str_b1").is_in(['11', '12', '13', '14', '15', '16', '17']),
  # ~pl.col("str_b1").is_in(['18', '19', '20', '21', '22', '23', '24', '26']),
  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_blast),
  pl.col("str_b5").is_in(possible_blast),
  ~pl.col("str_b5").is_in(['24', '38', '35']),
  pl.col("is_rare_combination") == False
)

df_possible

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""1112141531""","[11, 12, … 31]",11,12,14,15,31,"""11""","""12""","""14""","""15""","""31""","""11""","""30""",false,0,"""01_02_01_16""",18
"""1112141532""","[11, 12, … 32]",11,12,14,15,32,"""11""","""12""","""14""","""15""","""32""","""11""","""31""",false,0,"""01_02_01_17""",19
"""1112141533""","[11, 12, … 33]",11,12,14,15,33,"""11""","""12""","""14""","""15""","""33""","""11""","""32""",false,0,"""01_02_01_18""",20
"""1112141534""","[11, 12, … 34]",11,12,14,15,34,"""11""","""12""","""14""","""15""","""34""","""11""","""33""",false,0,"""01_02_01_19""",21
"""1112141731""","[11, 12, … 31]",11,12,14,17,31,"""11""","""12""","""14""","""17""","""31""","""11""","""30""",false,0,"""01_02_03_14""",10
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1728313234""","[17, 28, … 34]",17,28,31,32,34,"""17""","""28""","""31""","""32""","""34""","""17""","""33""",false,0,"""11_03_01_02""",-3
"""1728313334""","[17, 28, … 34]",17,28,31,33,34,"""17""","""28""","""31""","""33""","""34""","""17""","""35""",false,0,"""11_03_02_01""",-7
"""1729303233""","[17, 29, … 33]",17,29,30,32,33,"""17""","""29""","""30""","""32""","""33""","""17""","""35""",false,0,"""12_01_02_01""",-14


In [79]:
df_possible.sample(n=1)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""1522232932""","[15, 22, … 32]",15,22,23,29,32,"""15""","""22""","""23""","""29""","""32""","""15""","""31""",false,0,"""07_01_06_03""",-19
